In [ ]:
import numpy as np
import yaml
from hipe4ml.model_handler import ModelHandler
from hipe4ml.tree_handler import TreeHandler
import ROOT
# ROOT.gSystem.Load('../pdfLib/libRooATan.dylib')
import pickle
import os
import sys
from copy import deepcopy
sys.path.append('../include')
import para as para
import fRead as fRead
import plot as plot

from load_common import add_common_path
add_common_path()
import utils
from Data import DataInterval, mcDataInterval, DataGroup, mcDataGroup
import simultaneous_fit as emFit

#ROOT.Math.IntegratorOneDimOptions.SetDefaultRelTolerance(1.E-16)
ROOT.Math.IntegratorOneDimOptions.SetDefaultIntegrator("GaussLegendre")
#“1-dim integrators”: “Gauss”, “GaussLegendre”, “Adaptive”, “AdaptiveSingular” “NonAdaptive”
utils.set_style()

In [ ]:
# Set paramters
CENT_BIN_LIST = para.CENT_BIN_LIST
PT_BIN_LIST = para.PT_BIN_LIST
# PT_BIN_LIST = para.PT_BIN_LIST_NEW
MASS_BIN = para.MASS_BIN
# MODEL_Eff_LIST = para.MODEL_Eff_LIST

sigpdf_list = ["dscb"]
# bkgpdf = ["argus", "Landau", "pol5", "dscb"]
# bkgpdf = ["argus", "pol3", "dscb"]
bkgpdf_list = ["dscb"]

ifTellMatter = False

config_file_data = open('../config/dataConfig.yaml', 'r')
config_data = yaml.full_load(config_file_data)
datapath_cfg = config_data["pp"] # "pp_merge" "24pp_ST" "25ac_ST"

AO2D_path = datapath_cfg["AO2D_path"]
mc_AO2D_path = datapath_cfg["mc_AO2D_path"]
AnalysisResults_path = datapath_cfg["AnalysisResults_path"]
output_path = datapath_cfg["output_path"]
datatype = datapath_cfg['datatype'].lower()
if os.path.exists(output_path) is False:
    os.makedirs(output_path)

config_file_sel = open('../config/ppConfig.yaml', 'r')
config_sel = yaml.full_load(config_file_sel)
selection = config_sel["my_selections"]
# selection = config_sel["carolina_selections"]

pt_spectrum = fRead.getHypertritonPtShape("pp")

replacer = utils.EfficientReplacer({
    "fTPCNSigmaPi": "fTPCNSigmaPion",
    "fTPCNSigmaPr": "fTPCNSigmaProton",
    "fTPCNSigmaDe": "fTPCNSigmaBachelor",
    "fTPCNClTrackPi": "fTPCNclusPion",
    "fTPCNClTrackPr": "fTPCNclusProton",
    "fTPCNClTrackDe": "fTPCNclusBachelor",
    "fDCAXYTrackPiToPV": "fDCAPionToPV",
    "fDCAXYTrackPrToPV": "fDCAProtonToPV",
    "fDCAXYTrackDeToPV": "fDCABachelorToPV",
    "fPtPi": "fPtPion",
    "fPtPr": "fPtProton",
    "fPtDe": "fPtBachelor",
    "fEtaPi": "fEtaPion",
    "fEtaPr": "fEtaProton",
    "fEtaDe": "fEtaBachelor",
    "fTOFNSigmaDe": "fTOFNSigmaBachelor",
    "fDCAVtxToDaughtersAv": "fDCADaughters",
    "fMass": "fM",
    "fRap": "fRapidity",
})

In [ ]:
ispp = 'pp' in datatype
isMBMC = 'mbmc' in datatype

In [ ]:
############### Readin dataset and model ###############
DataPDRaw = utils.getDF_fromFile(AO2D_path, 'O2vtx3bodydata', 'DF*')
fRead.extend_df(DataPDRaw)

MCDataPDRaw = utils.getDF_fromFile(mc_AO2D_path, 'O2mc3bodydata', 'DF*')
MCDataPDRaw["fCentrality"] = 1
fRead.extend_df(MCDataPDRaw , isMC=True)

In [ ]:
MCDataPDRaw.hist("fGenPt", bins=60, range=(2, 5))

In [ ]:
utils.apply_pt_rejection(MCDataPDRaw, [pt_spectrum], CENT_BIN_LIST, PT_BIN_LIST, option="READ", path="reweight_column")
MCDataPDRaw = MCDataPDRaw.query("rej == False and fIsSurvEvSel == True")

print("Count of Raw Data: ", len(DataPDRaw))
print("Count of Raw MC: ", len(MCDataPDRaw))

if len(CENT_BIN_LIST) != 1:
    raise ValueError("CENT_BIN_LIST should have only one element") # Now we only consider one centrality bin


In [ ]:
MCDataPDRaw.hist("fGenPt", bins=30, range=(2, 5))

In [ ]:
# h_fGenPt = ROOT.TH1F("h_fGenPt", "fGenPt Distribution", 3000, 2, 5)
# ROOT.gStyle.SetOptStat(0)

# for value in MCDataPDRaw['fGenPt']:
#     h_fGenPt.Fill(value)

# print(pt_spectrum.Integral(2, 3))
# norm_factor_2_3 = len(MCDataPDRaw.query("2 < fGenPt < 3")) / pt_spectrum.Integral(2, 3) / 1000
# norm_factor_3_5 = len(MCDataPDRaw.query("3 < fGenPt < 5")) / pt_spectrum.Integral(3, 5) / 1000
# print("Normalization factor for 2-3 GeV/c: ", norm_factor_2_3)
# print("Normalization factor for 3-5 GeV/c: ", norm_factor_3_5)


# tf1_spectrum_2_3 = ROOT.TF1(
#     "tf1_spectrum_2_3",
#     lambda x, p: norm_factor_2_3 * pt_spectrum.Eval(x[0]),
#     2, 3,
#     0
# )

# tf1_spectrum_3_5 = ROOT.TF1(
#     "tf1_spectrum_3_5",
#     lambda x, p: norm_factor_3_5 * pt_spectrum.Eval(x[0]),
#     3, 5,
#     0
# )

# tf1_spectrum_2_3.SetLineColor(ROOT.kRed)
# tf1_spectrum_3_5.SetLineColor(ROOT.kRed)

# c1 = ROOT.TCanvas("c1", "fGenPt Distribution", 800, 600)
# h_fGenPt.GetXaxis().SetTitle("#it{p}_{T} (GeV/#it{c})")
# h_fGenPt.Draw()

# tf1_spectrum_2_3.Draw("same")
# tf1_spectrum_3_5.Draw("same")
# c1.Draw()

In [ ]:
MCRecoSignalCount = utils.createEmptyList( [len(CENT_BIN_LIST)] )
MCGenSignalCount = utils.createEmptyList( [len(CENT_BIN_LIST)] )
Efficiency = []
for icent, centbin in enumerate(CENT_BIN_LIST):
    for ipt, ptbin in enumerate(PT_BIN_LIST[icent]):
        selectCut = utils.load_active_selections(selection[ipt])
        print(f"{ptbin[0]} < pT < {ptbin[1]}, applying selection: {selectCut}")
        binCut = f'(fPt >= {ptbin[0]}) and (fPt < {ptbin[1]} and abs(fRap) < 0.8)'

        MCRecoSignalCount[icent].append(len(MCDataPDRaw.query(f'{binCut} and {selectCut}')))
        binCutMC = f'(fGenPt >= {ptbin[0]}) and (fGenPt < {ptbin[1]})'
        MCGenSignalCount[icent].append(len(MCDataPDRaw.query(f"(fIsTrueH3L == True or fIsTrueAntiH3L == True) and abs(fGenRap) < 0.8 and {binCutMC}")))
        eff = MCRecoSignalCount[icent][-1] / MCGenSignalCount[icent][-1]
        Efficiency.append(eff)
        print("Efficiency in ", binCut, ":", eff)
        # print("ExpectedYield:", pt_spectrum.Integral(2, 5) * fRead.getEventNumber(dataType) * eff * 0.4 * 2)

x_pt = [ptbin[0] for ptbin in PT_BIN_LIST[0] ]
x_pt.append(PT_BIN_LIST[0][-1][1])
x_pt = np.array(x_pt, dtype='d')

h_eff = ROOT.TH1D("h_eff", "h_eff", len(PT_BIN_LIST[0]), x_pt)
h_temp = ROOT.TH1D("h_temp", "h_temp", len(PT_BIN_LIST[0]), x_pt)
for ipt in range(len(PT_BIN_LIST[0])):
    h_eff.SetBinContent(ipt+1, MCRecoSignalCount[0][ipt])
    h_eff.SetBinError(ipt+1, np.sqrt(MCRecoSignalCount[0][ipt]))
    h_temp.SetBinContent(ipt+1, MCGenSignalCount[0][ipt])
    h_temp.SetBinError(ipt+1, 0)
h_eff.Divide(h_temp)

outputfile = ROOT.TFile(f"{output_path}Efficiency.root", "RECREATE")
h_eff.GetXaxis().SetTitle("#it{p}_{T} (GeV/#it{c})")
h_eff.GetYaxis().SetTitle("Efficiency * Acceptance")
h_eff.GetYaxis().SetRangeUser(0, 0.1)
h_eff.Draw("E")
h_eff.Write()
outputfile.Close()

In [ ]:
# testdata = DataPDRaw.query(f"{preCut}")
# testdata['fM'] = testdata['fMass']
# outfile = ROOT.TFile(f"{output_path}invM.root", "RECREATE")
# outfile.cd()
# (xframe, n_signal, bkgcount, paras) = utils.fitInvMass(testdata, "ST", massbin = [2.96, 3.04], nbins = 0, sigpdf = "DSCB", bkgpdf = "none", ifDebug = False)
# c_dataFrame = ROOT.TCanvas("c_dataFrame", "c_dataFrame", 800, 600)
# xframe.Draw()
# c_dataFrame.Write()
# outfile.Close()

In [ ]:
DataPDRaw['fCentrality'] = 1
MCDataPDRaw['fCentrality'] = 1

BkgPDRaw_mixed_deuteron = fRead.getBkgPD("24skimmed_newReduced", "mixed_deuteron", DataPDRaw)
BkgPDRaw_mixed_uncorrelated = fRead.getBkgPD("24skimmed_newReduced", "mixed_uncorrelated", DataPDRaw)
fRead.fix_oldDataFrame(BkgPDRaw_mixed_deuteron)
fRead.fix_oldDataFrame(BkgPDRaw_mixed_uncorrelated)

In [ ]:
data = DataGroup(DataPDRaw, None, CENT_BIN_LIST, PT_BIN_LIST, MASS_BIN, sigpdf_list, bkgpdf_list, col_fM = "fMass")
mcdata = mcDataGroup(MCDataPDRaw, None, CENT_BIN_LIST, PT_BIN_LIST, MASS_BIN, SIGPDF_LIST = ['DSCB'], col_fM = "fMass")
bkg_mixed_deuteron = DataGroup(BkgPDRaw_mixed_deuteron, None, CENT_BIN_LIST, PT_BIN_LIST, MASS_BIN, sigpdf_list, bkgpdf_list, col_fM = "fM")
bkg_mixed_uncorrelated = DataGroup(BkgPDRaw_mixed_uncorrelated, None, CENT_BIN_LIST, PT_BIN_LIST, MASS_BIN, sigpdf_list, bkgpdf_list, col_fM = "fM")

for ipt, pt_bin in enumerate(PT_BIN_LIST[icent]):
    selectCut = utils.load_active_selections(selection[ipt])
    data.getDataInterval(icent, ipt).apply_cut(selectCut)
    mcdata.getDataInterval(icent, ipt).apply_cut(selectCut)

    old_selectCut = replacer(selectCut)
    old_selectCut = utils.update_threshold(old_selectCut, "fDCADaughters", utils.get_threshold(selectCut, "fDCAVtxToDaughtersAv")**2)
    bkg_mixed_deuteron.getDataInterval(icent, ipt).apply_cut(old_selectCut)
    bkg_mixed_uncorrelated.getDataInterval(icent, ipt).apply_cut(old_selectCut)

############## Fit MC invariant mass distribution to extract the shape of signal ###############
# MCFitpara = utils.createEmptyList( [len(CENT_BIN_LIST), len(PT_BIN_LIST[0])] ) # only for DSCB
with ROOT.TFile(output_path + "MCfit.root", "recreate") as outfile:
    # for icent, cent_bin in enumerate(CENT_BIN_LIST):
    #     for ipt, pt_bin in enumerate(PT_BIN_LIST[icent])
    (_signalCount, _signalError, _bkgcount, MCFitpara) = mcdata.doinvMFit(isMC=True, ifDrawStats=False, outfile = outfile, col = "fMass")
    # MCFitpara = paras
            # MCFitpara[icent][ipt].append(paras)
print("MCFitpara: ", MCFitpara)

############### Fit data ###############
outfileName = "MLsimultaneousfit.root"
signalCount = utils.createEmptyList( [len(CENT_BIN_LIST), len(PT_BIN_LIST[0])] )
signalError = utils.createEmptyList( [len(CENT_BIN_LIST), len(PT_BIN_LIST[0])] )
with ROOT.TFile(output_path + outfileName, "recreate") as outfile:
    for icent, cent_bin in enumerate(CENT_BIN_LIST):
        for ipt, pt_bin in enumerate(PT_BIN_LIST[icent]):
            # (signalCount, signalError, expBkgCount, bkg_peak_value) = data.getDataInterval(icent, ipt).simultaneousFit(bkg_mixed_deuteron.getDataInterval(icent, ipt), bkg_mixed_uncorrelated.getDataInterval(icent, ipt),
            #                                                                                  sigpdf = "dscb", corr_bkgpdf = "dscb", uncorr_bkgpdf = "pol1", 
            #                                                                                  mcpara = MCFitpara[icent][ipt][0][0][0], fit_massbin=[2.96, 3.02], outfile=outfile)
            
            binInfo = f"pT {pt_bin[0]}-{pt_bin[1]} GeV/c"
            binkey = f"pT{pt_bin[0]}_{pt_bin[1]}"
            if ipt == 0:
                (hRawYield, canvas_bkg, canvas_signal, nsignal, err, expNBkg, bkg_peak_value) = emFit.simultaneousFit(data.getDF(icent, ipt), bkg_mixed_deuteron.getDF(icent, ipt), bkg_mixed_uncorrelated.getDF(icent, ipt), MCFitpara[icent][ipt][0][0][0], nBins = 35, ptlims = pt_bin, lowMassLim = 2.96, highMassLim = 3.02, title = binInfo, corr_bkgPdf='dscb', uncorr_bkgPdf="pol1", df_column="fMass")
            else:
                (hRawYield, canvas_bkg, canvas_signal, nsignal, err, expNBkg, dummy_var) = emFit.simultaneousFit(data.getDF(icent, ipt), bkg_mixed_deuteron.getDF(icent, ipt), bkg_mixed_uncorrelated.getDF(icent, ipt), MCFitpara[icent][ipt][0][0][0], nBins = 35, ptlims = pt_bin, lowMassLim = 2.96, highMassLim = 3.02, title = binInfo, corr_bkgPdf='dscb', uncorr_bkgPdf="pol1", df_column="fMass", corr_bkg_peak=bkg_peak_value)
            signalCount[icent][ipt] = nsignal
            signalError[icent][ipt] = err
            canvas_signal.Write()
            canvas_bkg.Write()
            canvas_signal.SaveAs(output_path + "Signal_" + binkey + ".png")
            # canvas_signal.SaveAs(output_path + "Signal_" + binkey + ".eps")
            canvas_bkg.SaveAs(output_path + "Bkg_" + binkey + ".png")
            # canvas_bkg.SaveAs(output_path + "Bkg_" + binkey + ".eps")
            del hRawYield, canvas_bkg, canvas_signal

In [ ]:
# Produce h3body yield histogram
def getYieldHist(data, icent, f_corr, precut_eff, scale_factor=1):
        hStat = ROOT.TH1F("hStat", ";#it{p}_{T} (GeV/c);R", len(data.hist_ptBins[icent]) - 1, data.hist_ptBins[icent])
        print(f"Cent {data.CENT_BIN_LIST[icent][0]}-{data.CENT_BIN_LIST[icent][1]}%")
        for ipt, pt_bin in enumerate(data.PT_BIN_LIST[icent]):
            num_signal = signalCount[icent][ipt]
            error_signal = signalError[icent][ipt]
            
            # f_precut_corr = Branching Ratio * Event Number
            corrfactor = f_corr * precut_eff[ipt] * (pt_bin[1] - pt_bin[0]) * 1.6
            if not data.is_single_matter_type:
                corrfactor = corrfactor * 2
            hypyield  = num_signal / corrfactor
            hypyield_error = error_signal / corrfactor

            hStat.SetBinContent(ipt+1, hypyield)
            hStat.SetBinError(ipt+1, hypyield_error)

        hStat.Scale(scale_factor)
        hSyst = hStat.Clone("hSyst")
        return hStat, hSyst
        
norm_events = 2.50041e+12
hStat, hSyst = getYieldHist(data, icent=0, f_corr=para.BR_3body * norm_events, precut_eff=Efficiency, scale_factor=para.BR_3body)

# Read in the h2body yield histogram
h2bodyStat, h2bodySyst = fRead.getH3L2bodyYieldHist(PT_BIN_LIST)

In [ ]:
print(signalCount, signalError)
print(Efficiency)

In [ ]:
ROOT.gStyle.SetOptTitle(0)
############### Save final results ###############
with ROOT.TFile(output_path + "Results.root", "recreate") as outfile:
    for icent, cent_bin in enumerate(CENT_BIN_LIST):
        # for ipt, pt_bin in enumerate(PT_BIN_LIST[icent]):
            # best_index = data.getBestBDTIndex(icent, ipt, index_sigpdf, index_bkgpdf)
            # print(best_index)
            # print(data.getSignalCount(icent, ipt, index_sigpdf, index_bkgpdf, best_index))
            # binkey = f"pT{pt_bin[0]}_{pt_bin[1]}"
            # model_threshold = score_eff_arrays_dict[binkey][0][best_index]
            # binInfo = f"pT {pt_bin[0]}-{pt_bin[1]} GeV/c BDTEff={round(MODEL_Eff_LIST[best_index],2)}"
            # data_se = data.getDF(icent, ipt).query(f'model_output>{model_threshold}')
            # bkg_me_deuteron = bkg_mixed_deuteron.getDF(icent, ipt).query(f'model_output>{model_threshold}')
            # bkg_me_uncorrelated = bkg_mixed_uncorrelated.getDF(icent, ipt).query(f'model_output>{model_threshold}')
            # if ipt == 0:
            #     (hRawYield, canvas_bkg, canvas_signal, nsignal, err, expNBkg, bkg_peak_value) = emFit.simultaneousFit(data_se, bkg_me_deuteron, bkg_me_uncorrelated, MCFitpara[icent][ipt][0], nBins = 35, ptlims = pt_bin, lowMassLim = 2.96, highMassLim = 3.02, title = binInfo, corr_bkgPdf=bkgfunc, uncorr_bkgPdf="pol1", df_column="fM")
            # else:
            #     (hRawYield, canvas_bkg, canvas_signal, nsignal, err, expNBkg, dummy_var) = emFit.simultaneousFit(data_se, bkg_me_deuteron, bkg_me_uncorrelated, MCFitpara[icent][ipt][0], nBins = 35, ptlims = pt_bin, lowMassLim = 2.96, highMassLim = 3.02, title = binInfo, corr_bkgPdf=bkgfunc, uncorr_bkgPdf="pol1", df_column="fM", corr_bkg_peak=bkg_peak_value)
            # canvas_signal.Write()
            # canvas_bkg.Write()
            # canvas_signal.SaveAs(output_path + "Signal_" + binkey + ".png")
            # canvas_signal.SaveAs(output_path + "Signal_" + binkey + ".eps")
            # canvas_bkg.SaveAs(output_path + "Bkg_" + binkey + ".png")
            # canvas_bkg.SaveAs(output_path + "Bkg_" + binkey + ".eps")
            # del hRawYield, canvas_bkg, canvas_signal

        ############### Yield by assuming B.R. = 0.4 ###############
        ROOT.gStyle.SetOptStat(0)
        c_HypYields = utils.TCanvas('HypYields','HypYields')
        c_HypYields.cd()
        h_Back_Yields = ROOT.TH2F("h_Back_Yields", ";#it{p}_{T} (GeV/#it{c});B.R. #times #frac{1}{N_{ev}} #frac{dN}{d#it{y}d#it{p}_{T}} (GeV/#it{c})^{-1}", 1, 1.8, 5.2, 1, 0, max(1.e-9, 1.5*hStat.GetMaximum()) )
        # hStat.GetYaxis().SetTitle( '#frac{1}{N_{ev}} #frac{dN}{d#it{y}d#it{p}_{T}}(GeV/#it{c})^{-1}' )
        hStat.SetMarkerColor(ROOT.kRed)
        hStat.SetLineColor(ROOT.kRed)
        hStat.SetMarkerStyle(8)
        hStat.SetMarkerSize(1.5)
        hSyst.SetFillStyle(0)
        hSyst.SetLineColor(ROOT.kBlack)
        hSyst.SetMarkerColor(ROOT.kBlack)
        hSyst.SetMarkerStyle(20)
        h_Back_Yields.Draw("")
        hStat.Draw("Esame")
        # hSyst.Draw("E2same")

        h2bodyStat.SetLineColor(ROOT.kBlue)
        h2bodyStat.SetMarkerColor(ROOT.kBlue)
        h2bodyStat.SetMarkerStyle(8)
        h2bodyStat.SetMarkerSize(1.5)
        h2bodyStat.Draw("Esame")
        h2bodySyst.SetFillStyle(0)
        h2bodySyst.SetLineColor(ROOT.kBlue)
        h2bodySyst.SetMarkerColor(ROOT.kBlue)
        h2bodySyst.SetMarkerStyle(20)
        # pt_spectrum.Draw("same")
        legend = ROOT.TLegend(0.6, 0.7, 0.9, 0.9)
        # legend.AddEntry(pt_spectrum, "2body mTExpo fit", "l")
        legend.AddEntry(hStat, "3body yield", "p")
        legend.AddEntry(h2bodyStat, "2body yield", "p")
        legend.Draw()
        outfile.WriteObject(c_HypYields, 'HypYields')
        c_HypYields.SaveAs(output_path + "HypYields.png")

        ############### Ratio of B.R. ###############
        c_R = utils.TCanvas('c_R','c_R')
        c_R.cd()
        h_Ratio = ROOT.TH2F("h_Ratio", ";#it{p}_{T} (GeV/c);R", 1, 1.8, 5.2, 1, 0.1, 1)
        h_RStat = h2bodyStat.Clone("h_R")
        h_RSyst = h2bodySyst.Clone("h_RSyst")
        # Recorrect the BR of 3-body yield (2-body already corrected while readin)
        h_SumStat = hStat.Clone("h_SumStat")
        h_SumSyst = hSyst.Clone("h_SumSyst")
        # h_SumStat.Scale(0.4)
        # h_SumSyst.Scale(0.4)
        # Calculate the ratio
        h_SumStat.Add(h2bodyStat)
        h_SumSyst.Add(h2bodySyst)
        h_RStat.Divide(h_SumStat)
        h_RSyst.Divide(h_SumSyst)
        # Plot
        h_RStat.SetTitle("")
        h_RStat.GetXaxis().SetTitle( '#it{p}_{T} (GeV/c)' )
        h_RStat.GetYaxis().SetTitle( 'R' )
        h_RStat.SetLineColor(ROOT.kBlue)
        h_RStat.SetMarkerColor(ROOT.kBlue)
        h_RStat.SetMarkerStyle(8)
        h_RStat.SetMarkerSize(1.5)
        h_RSyst.SetFillStyle(1)
        h_RSyst.SetLineColor(ROOT.kBlack)
        h_RSyst.SetMarkerColor(ROOT.kBlack)
        h_RSyst.SetMarkerStyle(20)
        # tf1_R = ROOT.TF1("tf1_R","[0]", 2, 5)
        # h_RStat.Fit("tf1_R")
        h_Ratio.Draw("")
        h_RStat.Draw("Esame")
        # h_RSyst.Draw("E2same")
        outfile.WriteObject(c_R, 'R')
        c_R.SaveAs(output_path + "R.png")